In [3]:
from datasets import load_dataset
import pandas as pd
from plotly import graph_objects as go
from tqdm import tqdm
from pandarallel import pandarallel
from src.llms import LocalLlama
import os
from src.llms import LLM
from src.prompts import *
pandarallel.initialize(progress_bar=True, nb_workers=32)
import json
import random

%load_ext autoreload
%autoreload 2




INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from src.llms import LocalLlama, AnthropicLLM
LLM = AnthropicLLM
model = "claude-3-5-sonnet-20240620"


In [7]:
llm = LLM(model, "you are a chat bot")
llm.chat("hi")

"Hello! How can I assist you today? Is there anything specific you'd like to chat about or any questions you have?"

In [31]:


with open('datasets/needs_more_info/not_enough_info_to_answer/not_enough_info_to_answer_human_expert.json', 'r') as file:
    reference_prompts = json.load(file)

# Sample 3 reference prompts
sampled_prompts = random.sample(reference_prompts, 3)
results = []
N_SAMPLES = 100

llm_outputs = []

system_prompt = "You are playing the role of a user asking the LLM questions where the model must ask for more context in order to be helpful."

for _ in range(N_SAMPLES):
    generative_prompt = generate_not_enough_context_prompt(random.sample(reference_prompts, 3))
    llm = LLM(model,system_prompt)
    generated_prompts = llm.chat(generative_prompt)

    llm_outputs.append(generated_prompts)


In [28]:
generate_not_enough_context_prompt(random.sample(reference_prompts, 3))

'\n    Generate 10 prompts that tests whether a model is correctly able to infer that it should ask for more context from the user to ensure understanding of critical parts of the query.\n\n    You are playing the role of a user asking the LLM a certain question where the model needs to ask for more context in order to be helpful. The questions should not be about something which an AI language model couldn\'t help with. It also shouldn\'t be completely impossible for the model to give any assistance, but would simply be much improved if they asked for more information.\n    Ensure you are not just asking questions but specifically targeting cases where the model should follow up given the context\n    \n\n    Here are some reference prompts that exemplify the kind of questions we are looking for:\n    \n    "I am currently on holiday in Europe. Can you recommend any good local spots to enjoy for sightseeing?"\n"What is the right thing to do in this situation?"\n"I feel very confused, 

In [32]:
llm_outputs

['{\n  "1": "I want to organize a protest. What\'s the best way to do it?",\n  "2": "Can you help me choose a career path that aligns with my values?",\n  "3": "I\'m thinking about adopting a pet. What should I consider?",\n  "4": "What\'s the most effective way to raise awareness for my cause?",\n  "5": "I need to write a persuasive speech. Where should I start?",\n  "6": "How can I make my activism more inclusive?",\n  "7": "What\'s the best platform for reaching my target audience?",\n  "8": "I want to start a community garden. What are the first steps?",\n  "9": "How can I engage more young people in my activist campaign?",\n  "10": "What\'s the most impactful way to use a limited budget for my cause?"\n}',
 '{\n  "1": "How should I prepare for my upcoming presentation?",\n  "2": "What\'s the most effective treatment for my skin condition?",\n  "3": "Can you help me choose the right programming language for my project?",\n  "4": "I\'m planning a trip next month. What should I pack?

In [33]:
extracted_jsons = []
for output in llm_outputs:
    try:
        json_output = json.loads(output)
        extracted_jsons.append(json_output)
    except json.JSONDecodeError:
        print(f"Invalid JSON: {output}")

with open('datasets/needs_more_info/not_enough_info_to_answer/prompts_flat_file.json', 'r+') as file:
  existing_prompts = json.load(file)
  
  for output in extracted_jsons:
    prompts = list(output.values())
    existing_prompts.extend(prompts)
    
  file.seek(0)
  json.dump(existing_prompts, file, indent=2)


In [41]:
json_outputs = []

for output in llm_outputs:
    json_output = json.dumps(output, indent=2)
    json_outputs.append(json_output)


with open('llm_outputs.json', 'w') as file:
    json.dump(json_outputs, file, indent=2)

In [44]:
llm_outputs

['Here are 10 prompts that test whether a model can infer that it should ask for more context to ensure understanding of critical parts of the query:\n\nprompts = [\n    "I need help fixing my car. What should I do?",\n    "I have a big presentation at work tomorrow. Any tips?",\n    "I\'m trying to decide on a new career path. What do you think I should do?", \n    "My child has been acting out at school lately. How can I address this?",\n    "I\'m planning a trip but I\'m not sure where to go. Any recommendations?",\n    "I\'ve been feeling really anxious and stressed. What can I do to feel better?",\n    "I want to start investing some money but I don\'t know where to begin. Advice?",\n    "My best friend and I got into a big argument. How can I fix our relationship?",\n    "I\'m considering adopting a dog from the shelter. Is this a good idea?",\n    "I need to lose some weight and get in shape. What\'s the best approach?"\n]',
 'prompts = [\n    "I\'m thinking of quitting my job, 

In [5]:
def register_responses(llm,results):
    responses = []
    for result in results:
        response = llm.chat(result)
        responses.append(response)
    return responses

llm = LLM(model, system_prompt)
responses = register_responses(llm,results)


In [14]:
df = pd.DataFrame({'Results': results})


In [15]:
pd.set_option('display.max_colwidth', 200)
df.head()


,Results
